In [5]:
# 问题：
#    1.分位数计算 --- 线性方法 （不同的方法会带来不同的结果）  但R语言应该采用的是linear方法 
#    2.分位数的值 --- 值的变化会对结果带来明显影响  ---  有一定可能R语言里的分位数和python中的分位数存在些许差异   但感觉可能性不大
#    3.R语言里的gev和lmoment  ---  感觉在python中的表示存在着一定的问题  想要着重考虑
# 解决方案：
#    1/2 --- 对应分位数选择和选取的方法
#    3 --- 对应lmoments3库

import pandas as pd
from scipy.stats import genextreme
from scipy import interpolate
import numpy as np
import lmoments3 as lm3
import lmoments as lm

def find_mf_func(stations_tab, amax_tab, start_year):
    megafloods_df = pd.DataFrame(columns=['id', 'LAT', 'LON', 'area', 'Q_MF', 'year_MF', 'date_MF'])
  
    nn = 0  # count number of outliers
    for i in range(len(stations_tab)):
        station_id = stations_tab['id'][i]
        dummy = amax_tab[amax_tab['id'] == station_id].dropna(subset=['q'])
        # dummy = amax_tab[amax_tab['id'] == station_id].dropna(subset=['q'])
        # dummy = amax_tab[amax_tab['id'] == station_id]
        
        if len(dummy) < 10:
            continue  # exclude short series
        
        # megafloods criteria:
        
        # dummy_quant = dummy['q'].quantile([0.25, 0.75])
        # dummy_upper = dummy_quant.loc[0.75] + 3 * (dummy_quant.loc[0.75] - dummy_quant.loc[0.25])
        # index_outlier = dummy[dummy['q'] >= dummy_upper].index

        # 1. outlier 
        dummy_sorted = sorted(dummy["q"])
        n = len(dummy_sorted)
        q1_index = int(0.25 * n) 
        q3_index = int(0.75 * n) 
        q1 = dummy_sorted[q1_index]
        q3 = dummy_sorted[q3_index]
        dummy_upper = q3 + 3 * (q3 - q1)
        # # 分位数计算，很怪  还行

        index_outlier = dummy[dummy['q'] >= dummy_upper].index
        if len(index_outlier) == 0:
            continue
        
        index_MF = []
        for tt in index_outlier:
            ty = dummy['year'][tt]
            # nyears_before = len(dummy[dummy['year'] < ty])   # 可能存在的问题 在下一行改了之后并没有明显影响
            nyears_before = len(dummy[dummy['year'] < ty]['q'])
            
            # 3. after start.year and with at least 20 years of data before event
            if nyears_before < 20 or ty < start_year:
                continue
            
        # # index_MF = []  # 存储异常事件的索引

        # # for tt in range(1, len(dummy)):
        #     qmax_old = dummy[dummy['year'] <= ty] ['q'] # 根据年份选择流量值的子集

        # # # 计算L-moments
        # # lmoments = lm3.lmom_ratios(qmax_old.values)

        # # 使用L-moments进行GEV分布拟合
        #     params = lm.pelgev(qmax_old)
        

        # # 使用pextRemes计算概率
        #     neP = genextreme.cdf(qmax_old,*params)
        #     RP = 1 / (1 - neP)
        #     RP[np.isinf(RP)] = 10**5
        #     rRP = RP[-1] / max(RP[:-1])

        #     if rRP > 3:
        #         index_MF.append(tt)

        # index_outlier = index_MF[1:]

            # 2. surprising (Tl/Tsl > 3) record-breaking event
            qmax_old = dummy[dummy['year'] <= ty]['q']    # 同上的问题
            # 计算概率分布函数
           

            local_gev = genextreme.fit(qmax_old)
            neP = 1-genextreme.pdf(qmax_old, *local_gev)   # 方法计算对吗  这是真的大问题
           
            RP = 1 / (1 - neP)
            RP[RP == float('inf')] = 10 ** 5
            rRP = RP[-1] / max(RP[:-1])
            if rRP > 3:
                index_MF.append(tt)
                
        index_outlier = index_MF
        if len(index_MF) > 0:
            for oo in index_MF:
                nn += 1
                try:
                    megafloods_df.loc[nn] = [stations_tab['id'][i], stations_tab['lat'][i], stations_tab['lon'][i],
                                         float(stations_tab['area'][i]), dummy['q'][oo], dummy['year'][oo],
                                         pd.to_datetime(
                                             f"{int(dummy['year'][oo])}-{int(dummy['month'][oo])}-{int(dummy['day'][oo])}")]
                except ValueError:
                    megafloods_df.loc[nn] = [stations_tab['id'][i], stations_tab['lat'][i], stations_tab['lon'][i],
                                         float(stations_tab['area'][i]), dummy['q'][oo], dummy['year'][oo],-1]
    
    return megafloods_df

stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")
megafloods_df = find_mf_func(stations,flood_data,2000)
# print(find_mf_func(stations,flood_data,2000))

In [6]:
# 第二部分
# 1.平均值和最大值
import pandas as pd

stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")


# 计算平均值和最大值
mean_q = flood_data.iloc[:, 1].groupby(flood_data['id']).mean().round(5)
max_q = flood_data.iloc[:, 1].groupby(flood_data['id']).max()

# 将结果赋给 stations 数据框
stations['mean_q'] = stations['id'].map(mean_q)
stations['max_q'] = stations['id'].map(max_q)

# print(stations)

In [7]:
# 第二部分1：五个区域的包络曲线
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

def env_func(stations_tab, amax_tab):
    # Initialize an empty DataFrame for the results
    columns = ['slope_qr_median', 'intercept_qr_median', 'slope_qr_env', 'intercept_qr_env', 'intercept_env', 'n_stat_years']
    ENV_tab = pd.DataFrame(columns=columns, index=range(1, 6))
    
    for rr in range(1, 6):
        # print(rr)     打印这个东西干嘛?

        # Filter stations and amax data by region
        dummy_stations = stations_tab[stations_tab['regions1'] == rr]
        dummy_q = amax_tab[amax_tab['id'].isin(dummy_stations['id'])]
        dummy_q = dummy_q[dummy_q['q'] > 0]
        dummy_q['qlog10'] = np.log10(dummy_q['q'])
        dummy_q['Alog10'] = np.nan

        # Assign area log10 values
        for i in dummy_stations['id']:
            dummy_q.loc[dummy_q['id'] == i, 'Alog10'] = np.log10(dummy_stations.loc[dummy_stations['id'] == i, 'area'].values[0])

        # Quantile regression for tau=0.5
        model_median = smf.quantreg('qlog10 ~ Alog10', dummy_q).fit(q=0.5)
        ENV_tab.at[rr, 'slope_qr_median'] = model_median.params['Alog10']
        ENV_tab.at[rr, 'intercept_qr_median'] = model_median.params['Intercept']

        # Quantile regression for tau=0.999
        model_env = smf.quantreg('qlog10 ~ Alog10', dummy_q).fit(q=0.999)
        ENV_tab.at[rr, 'slope_qr_env'] = model_env.params['Alog10']
        ENV_tab.at[rr, 'intercept_qr_env'] = model_env.params['Intercept']

        # Calculate intercept of envelope
        intercept = -ENV_tab.at[rr, 'slope_qr_env'] * np.log10(dummy_stations['area']) + np.log10(dummy_stations['max_q'])
        ENV_tab.at[rr, 'intercept_env'] = intercept.max()

        # Number of station years
        ENV_tab.at[rr, 'n_stat_years'] = len(dummy_q)

    return ENV_tab
# stations = pd.read_csv("./gauges.csv")
# flood_data = pd.read_csv("./floods.csv")
# print(env_func(stations, flood_data))
ENV_reg = env_func(stations, flood_data)



In [8]:
# 第二部分2：EDT1 and EDT2
# 未订正
import pandas as pd
import numpy as np
from math import log10

# EDT 1
def ext_data_tab1_func(envelope_tab):
    ExtDataTable1 = pd.DataFrame(np.nan, index=envelope_tab.index, columns=["Slope_Envelope","Intercept_Envelope","Intercept_Envelope_1000km2","Slope_median_regression","Intercept_median_regression"])
    ExtDataTable1["Slope_Envelope"] = envelope_tab["slope_qr_env"]
    ExtDataTable1["Intercept_Envelope"] = envelope_tab["intercept_env"]
    ExtDataTable1["Intercept_Envelope_1000km2"] = 10 ** (envelope_tab["intercept_env"] + envelope_tab["slope_qr_env"] * log10(1000))
    ExtDataTable1["Slope_median_regression"] = envelope_tab["slope_qr_median"]
    ExtDataTable1["Intercept_median_regression"] = envelope_tab["intercept_qr_median"]
    ExtDataTable1 = ExtDataTable1.round(2)
    return ExtDataTable1

ExtDataTable1 = ext_data_tab1_func(ENV_reg)
print(ExtDataTable1)


  Slope_Envelope Intercept_Envelope Intercept_Envelope_1000km2  \
1      -0.074694           0.359792                   1.366811   
2      -0.382777           1.485215                   2.172128   
3      -0.566036           2.419434                   5.264131   
4      -0.270014           1.487376                   4.756991   
5      -0.337627           1.486842                   2.978257   

  Slope_median_regression Intercept_median_regression  
1               -0.091551                   -0.879714  
2               -0.217651                   -0.546294  
3               -0.284169                   -0.021829  
4               -0.199583                   -0.177523  
5               -0.146376                   -0.647347  


In [9]:
# EDT 2
ExtDataTable2 = pd.DataFrame(np.nan, index=range(1, 6), columns=["Number_of_gauges","Avg_Ratio_max_mean_Q","Number_of_targets", "Percentage_of_targets","Avg_Ratio_max_mean_Q_targets"])
for rr in range(1, 6):
    dummy_stations = stations[stations["regions1"] == rr]
    ExtDataTable2.loc[rr, "Number_of_gauges"] = dummy_stations.shape[0]
    ExtDataTable2.loc[rr, "Avg_Ratio_max_mean_Q"] = (dummy_stations["max_q"] / dummy_stations["mean_q"]).mean().round(2)
    dummy_mf = dummy_stations[dummy_stations["id"].isin(megafloods_df["id"])]
    ExtDataTable2.loc[rr, "Number_of_targets"] = dummy_mf.shape[0]
    ExtDataTable2.loc[rr, "Percentage_of_targets"] = (ExtDataTable2.loc[rr, "Number_of_targets"] * 100 / ExtDataTable2.loc[rr, "Number_of_gauges"]).round(1)
    ExtDataTable2.loc[rr, "Avg_Ratio_max_mean_Q_targets"] = (dummy_mf["max_q"] / dummy_mf["mean_q"]).mean().round(2)

del dummy_mf, dummy_stations

print(ExtDataTable2)

   Number_of_gauges  Avg_Ratio_max_mean_Q  Number_of_targets  \
1             671.0                  2.09                7.0   
2            3660.0                  3.25              262.0   
3             938.0                  3.50               21.0   
4            1240.0                  2.83               80.0   
5            1514.0                  2.36              121.0   

   Percentage_of_targets  Avg_Ratio_max_mean_Q_targets  
1                    1.0                          2.92  
2                    7.2                          5.40  
3                    2.2                          5.39  
4                    6.5                          4.03  
5                    8.0                          3.44  


In [10]:
print(stations)

        id    lat    lon     area  regions1  regions2   mean_q    max_q
0        1  40.61  19.66   6708.0         3         3  0.23901  0.58069
1        2  48.40  15.44  94762.0         2         4  0.05648  0.11537
2        3  47.29   9.68   4626.0         4         4  0.20537  0.30831
3        4  47.02   9.92    145.0         4         4  0.13750  0.26087
4        5  47.04   9.95     39.3         4         4  0.31597  1.15013
...    ...    ...    ...      ...       ...       ...      ...      ...
8018  8019  46.93  36.86   1627.0         2         2  0.01490  0.13580
8019  8020  47.13  37.59   3722.0         2         2  0.01724  0.15784
8020  8021  47.46  37.45    194.0         2         2  0.03520  0.48969
8021  8022  47.25  37.23    269.0         2         2  0.06849  0.51111
8022  8023  48.10  38.11    272.0         2         2  0.04906  0.20588

[8023 rows x 8 columns]


In [31]:
# 第三部分:try2 
# 似乎流程正确，但总感觉差点什么（没有完全显示而已）
import pandas as pd
import numpy as np
import math

def mhq_cv_func(stations_df, amax_df, target_years):
    EU_MHQ_CV_list = {}
    for target_year in target_years:
        # print(target_year)
        EU_MHQ_CV = pd.DataFrame(columns=["MHQ", "CV", "MHQ_100"])
        EU_MHQ_CV = pd.concat([stations_df[['id', 'lat', 'lon', 'area']], EU_MHQ_CV], axis=1)
        for index, station in EU_MHQ_CV.iterrows():
            dummy_df = amax_df[(amax_df['id'] == station['id']) & (amax_df['year'] < target_year)]
            dummy_df = dummy_df.dropna(subset=['q'])
            # print(f'Year: {target_year}, Station ID: {station["id"]}, Dummy DF Length: {len(dummy_df)}')
            if len(dummy_df) < 10: 
                continue
            dummy_Q = dummy_df['q'].values
            dummy_Y = dummy_df['year'].values
            # Exclude outliers
            Q1 = np.quantile(dummy_Q, 0.25)
            Q3 = np.quantile(dummy_Q, 0.75)
            IQR = Q3 - Q1
            upper_bound = Q3 + 3 * IQR
            filtered_indices = dummy_Q <= upper_bound
            dummy_Q = dummy_Q[filtered_indices]
            dummy_Y = dummy_Y[filtered_indices]
            EU_MHQ_CV.at[index, 'MHQ'] = np.mean(dummy_Q)
            EU_MHQ_CV.at[index, 'CV'] = np.std(dummy_Q) / EU_MHQ_CV.at[index, 'MHQ']
        # Normalize MHQ to 100km2
        EU_MHQ_CV['MHQ'] = EU_MHQ_CV['MHQ'].replace(0, np.nan)  # Replace 0 with NaN to avoid log(0) errors
        EU_MHQ_CV['MHQ_log'] = np.log10(EU_MHQ_CV['MHQ'].astype(float))  # Corrected
        stations_df = stations_df.dropna(subset=['area'])
        EU_MHQ_CV['area_log'] = np.log10(stations_df['area'].astype(float))  # Corrected
        area_regr = np.polyfit(EU_MHQ_CV['area_log'], EU_MHQ_CV['MHQ_log'], 1)
        EU_MHQ_CV['MHQ_100'] = 10 ** (EU_MHQ_CV['MHQ_log'] - area_regr[0] * EU_MHQ_CV['area_log'] + area_regr[0] * np.log10(100))
        # 删除还有NaN的行
        # EU_MHQ_CV = EU_MHQ_CV.dropna(subset=['area', 'MHQ_100', 'CV'])
        EU_MHQ_CV_list[target_year] = EU_MHQ_CV.drop(columns=['MHQ_log', 'area_log'])
    return EU_MHQ_CV_list


EU_MHQ_CV_list = mhq_cv_func(stations, flood_data, range(2000, 2023))
print(EU_MHQ_CV_list)


{2000:         id    lat    lon     area       MHQ        CV  MHQ_100
0        1  40.61  19.66   6708.0  0.239013  0.471835      NaN
1        2  48.40  15.44  94762.0  0.055616  0.278194      NaN
2        3  47.29   9.68   4626.0       NaN       NaN      NaN
3        4  47.02   9.92    145.0  0.127732  0.412555      NaN
4        5  47.04   9.95     39.3  0.329941  0.383054      NaN
...    ...    ...    ...      ...       ...       ...      ...
8018  8019  46.93  36.86   1627.0  0.011363  1.238913      NaN
8019  8020  47.13  37.59   3722.0  0.015584  0.851532      NaN
8020  8021  47.46  37.45    194.0  0.024747  0.960986      NaN
8021  8022  47.25  37.23    269.0  0.053101  0.958687      NaN
8022  8023  48.10  38.11    272.0  0.061789  0.849174      NaN

[8023 rows x 7 columns], 2001:         id    lat    lon     area       MHQ        CV  MHQ_100
0        1  40.61  19.66   6708.0  0.239013  0.471835      NaN
1        2  48.40  15.44  94762.0  0.055523  0.277902      NaN
2        3  47.2

In [12]:
print(megafloods_df)

         id    LAT    LON    area     Q_MF  year_MF              date_MF
1      10.0  47.17   9.78    95.5  1.15183   2005.0  2005-08-23 00:00:00
2      11.0  47.29   9.85    33.4  7.84431   2005.0  2005-08-22 00:00:00
3      14.0  47.26   9.96    41.7  3.83693   2005.0  2005-08-23 00:00:00
4      15.0  47.23   9.78   148.0  2.40455   2005.0  2005-08-23 00:00:00
5      17.0  47.36  10.19    31.1  3.89068   2005.0  2005-08-23 00:00:00
..      ...    ...    ...     ...      ...      ...                  ...
503  7759.0  48.69  23.48     3.2  1.38125   2015.0  2015-11-21 00:00:00
504  7843.0  49.14  23.03   137.0  1.95588   2008.0  2008-07-25 00:00:00
505  7855.0  49.41  22.93   384.0  1.87760   2008.0  2008-07-25 00:00:00
506  7860.0  48.21  24.98  1501.0  1.00000   2008.0  2008-07-26 00:00:00
507  7862.0  48.56  24.76    18.1  4.79006   2006.0  2006-06-30 00:00:00

[507 rows x 7 columns]


In [55]:
# 第三部分：预测
# 问题：没有用到“target_years”输入必定会少东西的咯
# 当前：存在时间格式上的问题
import pandas as pd
import numpy as np
from scipy.stats import zscore
import datetime

def predict_mf_function(mf_list, stations_tab, target_years, amax_tab, mhq_cv_tab, env_tab):
    w1, w2, w3 = 1, 1, 1
    DIST = 1

    REG_MEGAFLOODS_list = []
    REG_ENVELOPE_param = []
    REG_DONORS_list = []

    for REG in range(1, 6):  # loop on Regions
        REG_list_stations = stations_tab[stations_tab['regions1'] == REG]['id']
        MEGAFLOODS_list = mf_list[mf_list['id'].isin(REG_list_stations)]
        DONORS_list = []
        ENVELOPE_param = pd.DataFrame(columns=["slope", "intercept.region", "intercept.mf", "q.predicted", "n_statyears_donors", "n_stat_donors"])

        for i, row in MEGAFLOODS_list.iterrows():  # loop on megafloods
            TARGET_YEAR = row['year_MF']
            TARGET_Q = row['Q_MF']
            TARGET_DATE = row['date_MF']
            code_mf = row['id']
            EU_MHQ_CV = mhq_cv_tab[TARGET_YEAR].dropna(subset=['MHQ'])
            # print(EU_MHQ_CV)
            # EU_MHQ_CV = EU_MHQ_CV[~EU_MHQ_CV['MHQ'].isna()]
            EU_MHQ_CV = EU_MHQ_CV[EU_MHQ_CV['id'].isin(REG_list_stations)]
    
            # distances
            ind_stat = EU_MHQ_CV[EU_MHQ_CV['id'] == code_mf].index[0]
            # sd1 = zscore(np.log10(EU_MHQ_CV['area']))
            # sd2 = zscore(np.log10(EU_MHQ_CV['MHQ_100']))
            # sd3 = zscore(np.log10(EU_MHQ_CV['CV']))
            # 先计算 log10
            log_area = np.log10(EU_MHQ_CV['area'])
            log_mhq_100 = np.log10(EU_MHQ_CV['MHQ_100'])
            # 处理 NaN 值
            EU_MHQ_CV['CV'].fillna(0, inplace=True)
            log_cv = np.log10(EU_MHQ_CV['CV'].astype(float))

            # 然后计算 zscore
            sd1 = zscore(log_area)
            sd2 = zscore(log_mhq_100)
            sd3 = zscore(log_cv)

            d1 = sd1 - sd1[ind_stat]
            d2 = sd2 - sd2[ind_stat]
            d3 = sd3 - sd3[ind_stat]
            dd = np.sqrt((d1**2)*w1 + (d2**2)*w2 + (d3**2)*w3)
            # donor group
            group = EU_MHQ_CV['id'][dd <= DIST].values
            stations_Reg_group = stations_tab[stations_tab['id'].isin(group)]
            EU_MHQ_CV_group = EU_MHQ_CV[EU_MHQ_CV['id'].isin(group)]
            # characteristics of donor catchments
            DONORS = pd.DataFrame(columns=["id", "lat", "lon", "area", "SFOR.B", "SFOR.year.B", "SFOR.date.B"])

            for s, row_group in EU_MHQ_CV_group.iterrows():
                dummy = amax_tab[amax_tab['id'] == row_group['id']]
                QQ = dummy['q'][dummy['year'] < TARGET_YEAR]
                DD = pd.to_datetime(dummy[['year', 'month', 'day']][dummy['year'] < TARGET_YEAR])
                YY = dummy['year'][dummy['year'] < TARGET_YEAR]
                ind_SFOR = QQ.idxmax()
                # DONORS = DONORS.append({
                #     "id": row_group['id'],
                #     "lat": row_group['lat'],
                #     "lon": row_group['lon'],
                #     "area": row_group['area'],
                #     "SFOR.B": QQ.loc[ind_SFOR],
                #     "SFOR.year.B": YY.loc[ind_SFOR],
                #     "SFOR.date.B": DD.loc[ind_SFOR]
                # }, ignore_index=True)

                # 假设row_group包含要添加的数据
                new_data = {
                    "id": row_group['id'],
                    "lat": row_group['lat'],
                    "lon": row_group['lon'],
                    "area": row_group['area'],
                    "SFOR.B": QQ.loc[ind_SFOR],
                    "SFOR.year.B": YY.loc[ind_SFOR],
                    "SFOR.date.B": DD.loc[ind_SFOR]
                }

                # 创建包含新数据的DataFrame
                new_df = pd.DataFrame(new_data, index=[0])

                # 使用pd.concat将新数据添加到DONORS中
                DONORS = pd.concat([DONORS, new_df], ignore_index=True)


            # envelope with region slope
            slope = env_tab['slope_qr_env'][REG]
            intercept_region = env_tab['intercept_qr_env'][REG]
            intercept = -slope * np.log10(DONORS['area']) + np.log10(DONORS['SFOR.B'])
            DONORS['intercept'] = intercept
            DONORS = DONORS.sort_values(by='intercept', ascending=False)
            DONORS_list.append(DONORS)

            # 假设要添加的数据为new_data
            new_df = pd.DataFrame({
                "slope": slope,
                "intercept.region": intercept_region,
                "intercept.mf": intercept.max(),
                "q.predicted": intercept.max() + slope * np.log10(row['area']),
                "n_statyears_donors": amax_tab[amax_tab['id'].isin(DONORS['id'])].shape[0],
                "n_stat_donors": len(group)
            }, index=[0])

            # 使用pd.concat将新数据添加到ENVELOPE_param中
            ENVELOPE_param = pd.concat([ENVELOPE_param, new_df], ignore_index=True)

            # ENVELOPE_param = ENVELOPE_param.append({
            #     "slope": slope,
            #     "intercept.region": intercept_region,
            #     "intercept.mf": intercept.max(),
            #     "q.predicted": intercept.max() + slope * np.log10(row['area']),
            #     "n_statyears_donors": amax_tab[amax_tab['id'].isin(DONORS['id'])].shape[0],
            #     "n_stat_donors": len(group)
            # }, ignore_index=True)

        REG_MEGAFLOODS_list.append(MEGAFLOODS_list)
        REG_ENVELOPE_param.append(ENVELOPE_param)
        REG_DONORS_list.append(DONORS_list)

    RES = {
        "mf.list": REG_MEGAFLOODS_list,
        "EC.param": REG_ENVELOPE_param,
        "donor.list": REG_DONORS_list
    }
    return RES

# 示例数据加载和函数调用部分
# 请确保你已经准备好了mf_list, stations_tab, target_years, amax_tab, mhq_cv_tab, env_tab这些变量的数据
# 例如:
# mf_list = pd.read_csv('megafloods.csv')
# stations_tab = pd.read_csv('stations.csv')
# target_years = np.array([2000, 2001, 2002, ..., 2022])
# amax_tab = pd.read_csv('flood_data.csv')
# mhq_cv_tab = {2000: pd.DataFrame(...), 2001: pd.DataFrame(...), ...}
# env_tab = pd.read_csv('ENV_reg.csv')

MF_predicted = predict_mf_function(megafloods_df, stations, np.array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]), flood_data, EU_MHQ_CV_list, ENV_reg)



C:\Users\hanchan\AppData\Local\Temp\ipykernel_19020\3713120121.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ENVELOPE_param = pd.concat([ENVELOPE_param, new_df], ignore_index=True)
C:\Users\hanchan\AppData\Local\Temp\ipykernel_19020\3713120121.py:113: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ENVELOPE_param = pd.concat([ENVELOPE_param, new_df], ignore_index=True)
C:\Users\hanchan\AppData\Local\Temp\ipykernel_19020\3713120121.py:91: FutureWarning: The behavior of DataFrame concatenati

ValueError: cannot assemble the datetimes: unconverted data remains when parsing with format "%Y%m%d": "05", at position 3. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [23]:
# 同上问题：没有用到“target_years”输入必定会少东西的咯
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime

def predict_mf_function(mf_list, stations_tab, target_years, amax_tab, mhq_cv_tab, env_tab):
    w1 = 1
    w2 = 1
    w3 = 1
    DIST = 1

    REG_MEGAFLOODS_list = [None]*5
    REG_ENVELOPE_param = [None]*5
    REG_DONORS_list = [None]*5

    for REG in range(5):
        REG_list_stations = stations_tab['id'][stations_tab['regions1'] == REG]
        MEGAFLOODS_list = mf_list[mf_list['id'].isin(REG_list_stations)]
        DONORS_list = [None]*len(MEGAFLOODS_list)
        ENVELOPE_param = pd.DataFrame(np.nan, index=range(len(MEGAFLOODS_list)), columns=['slope', 'intercept.region', 'intercept.mf', 'q.predicted', 'n_statyears_donors', 'n_stat_donors'])

        for i in range(len(MEGAFLOODS_list)):
            print(i)

            TARGET_YEAR = MEGAFLOODS_list['year_MF'].iloc[i]
            TARGET_Q = MEGAFLOODS_list['Q_MF'].iloc[i]
            TARGET_DATE = MEGAFLOODS_list['date_MF'].iloc[i]
            code_mf = MEGAFLOODS_list['id'].iloc[i]
            EU_MHQ_CV = mhq_cv_tab[TARGET_YEAR]
            EU_MHQ_CV = EU_MHQ_CV[~EU_MHQ_CV['MHQ'].isna()]
            EU_MHQ_CV = EU_MHQ_CV[EU_MHQ_CV['id'].isin(REG_list_stations)]

            ind_stat = EU_MHQ_CV['id']
            sd1 = (np.log10(EU_MHQ_CV['area']) - np.mean(np.log10(EU_MHQ_CV['area']))) / np.std(np.log10(EU_MHQ_CV['area']))
            sd2 = (np.log10(EU_MHQ_CV['MHQ_100']) - np.mean(np.log10(EU_MHQ_CV['MHQ_100']))) / np.std(np.log10(EU_MHQ_CV['MHQ_100']))
            sd3 = (np.log10(EU_MHQ_CV['CV']) - np.mean(np.log10(EU_MHQ_CV['CV']))) / np.std(np.log10(EU_MHQ_CV['CV']))
            d1 = sd1 - sd1[ind_stat]
            d2 = sd2 - sd2[ind_stat]
            d3 = sd3 - sd3[ind_stat]
            dd = np.sqrt((d1**2)*w1 + (d2**2)*w2 + (d3**2)*w3)
            group = EU_MHQ_CV['id'][dd <= DIST]
            stations_Reg_group = stations_tab[stations_tab['id'].isin(group)]
            EU_MHQ_CV_group = EU_MHQ_CV[EU_MHQ_CV['id'].isin(group)]

            DONORS = pd.DataFrame(columns=['id', 'lat', 'lon', 'area', 'SFOR.B', 'SFOR.year.B', 'SFOR.date.B'])
            for s in range(len(EU_MHQ_CV_group)):
                DONORS.loc[s, 'id'] = EU_MHQ_CV_group['id'].iloc[s]
                DONORS.loc[s, 'lat'] = EU_MHQ_CV_group['lat'].iloc[s]
                DONORS.loc[s, 'lon'] = EU_MHQ_CV_group['lon'].iloc[s]
                DONORS.loc[s, 'area'] = EU_MHQ_CV_group['area'].iloc[s]
                dummy = amax_tab[amax_tab['id'] == EU_MHQ_CV_group['id'].iloc[s]]
                QQ = dummy['q'][dummy['year'] < TARGET_YEAR]
                DD = [datetime.strptime(f"{y}-{m}-{d}", "%Y-%m-%d") for y, m, d in zip(dummy['year'][dummy['year'] < TARGET_YEAR], dummy['month'][dummy['year'] < TARGET_YEAR], dummy['day'][dummy['year'] < TARGET_YEAR])]
                YY = dummy['year'][dummy['year'] < TARGET_YEAR]
                ind_SFOR = np.argmax(QQ)
                if len(ind_SFOR) > 1:
                    ind_SFOR = np.max(ind_SFOR)
                DONORS.loc[s, 'SFOR.B'] = QQ[ind_SFOR]
                DONORS.loc[s, 'SFOR.year.B'] = YY[ind_SFOR]
                DONORS.loc[s, 'SFOR.date.B'] = DD[ind_SFOR]

            ENVELOPE_param['slope'].iloc[i] = env_tab['slope_qr_env'].iloc[REG]
            ENVELOPE_param['intercept.region'].iloc[i] = env_tab['intercept_qr_env'].iloc[REG]
            intercept = -ENVELOPE_param['slope'].iloc[i]*np.log10(DONORS['area']) + np.log10(DONORS['SFOR.B'])
            DONORS = DONORS.sort_values(by='intercept', ascending=False)
            DONORS_list[i] = DONORS
            ENVELOPE_param['intercept.mf'].iloc[i] = np.max(intercept)
            ENVELOPE_param['q.predicted'].iloc[i] = ENVELOPE_param['intercept.mf'].iloc[i] + ENVELOPE_param['slope'].iloc[i]*np.log10(MEGAFLOODS_list['area'].iloc[i])
            ENVELOPE_param['n_statyears_donors'].iloc[i] = len(amax_tab[amax_tab['id'].isin(DONORS['id'])])
            ENVELOPE_param['n_stat_donors'].iloc[i] = len(group)

        REG_MEGAFLOODS_list[REG] = MEGAFLOODS_list
        REG_ENVELOPE_param[REG] = ENVELOPE_param
        REG_DONORS_list[REG] = DONORS_list

    RES = {'mf.list': REG_MEGAFLOODS_list, 'EC.param': REG_ENVELOPE_param, 'donor.list': REG_DONORS_list}
    return RES

MF_predicted = predict_mf_function(megafloods_df, stations,2000, flood_data, EU_MHQ_CV_list, ENV_reg)
print(MF_predicted)


0


KeyError: "None of [Index([ 842,  843,  844,  845,  846,  850,  851,  853,  855,  856,\n       ...\n       6929, 6930, 6931, 6933, 6934, 6935, 6936, 6937, 6938, 6939],\n      dtype='int64', length=666)] are in the [index]"

In [ ]:
# 自己研究 不完整
import pandas as pd
import numpy as np
from sklearn.linear_model import QuantileRegressor

def predict_mf_function(mf_list, stations_tab, target_years, amax_tab, mhq_cv_tab, env_tab):

    # Specify weights and distance limit for finding donor group
    w1 = 1
    w2 = 1
    w3 = 1
    DIST = 1
    
    # Prepare lists for saving results
    reg_megafloods_list = [None] * 5  # List megafloods per each region
    reg_envelope_param = [None] * 5  # Envelope parameters
    reg_donors_list = [None] * 5  # Donor list per region per megaflood

    for REG in range(5):  # loop on Regions
        reg_list_stations = stations_tab.loc[stations_tab['regions1'] == REG, 'id']
        megafloods_list = mf_list[mf_list['id'].isin(reg_list_stations)]
        donors_list = []
        envelope_param = pd.DataFrame(np.nan, index=range(len(megafloods_list)), columns=['slope', 'intercept.region', 'intercept.mf', 'q.predicted', 'n_statyears_donors', 'n_stat_donors'])

        # Loop on megafloods
        for i in range(len(megafloods_list)):
            print(i)

            target_year = megafloods_list.iloc[i]['year_MF']
            target_q = megafloods_list.iloc[i]['Q_MF']
            target_date = megafloods_list.iloc[i]['date_MF']
            code_mf = megafloods_list.iloc[i]['id']

            eu_mhq_cv = mhq_cv_tab[target_years.index(target_year)]
            eu_mhq_cv = eu_mhq_cv.dropna(subset=['MHQ'])
            eu_mhq_cv = eu_mhq_cv[eu_mhq_cv['id'].isin(reg_list_stations)]

            # distances
            ind_stat = eu_mhq_cv[eu_mhq_cv['id'] == code_mf].index[0]
            sd1 = (np.log10(eu_mhq_cv['area']) - np.mean(np.log10(eu_mhq_cv['area']))) / np.std(np.log10(eu_mhq_cv['area']))
            sd2 = (np.log10(eu_mhq_cv['MHQ_100']) - np.mean(np.log10(eu_mhq_cv['MHQ_100']))) / np.std(np.log10(eu_mhq_cv['MHQ_100']))
            sd3 = (np.log10(eu_mhq_cv['CV']) - np.mean(np.log10(eu_mhq_cv['CV']))) / np.std(np.log10(eu_mhq_cv['CV']))

            d1 = sd1 - sd1[ind_stat]
            d2 = sd2 - sd2[ind_stat]
            d3 = sd3 - sd3[ind_stat]

            dd = np.sqrt((d1**2)*w1 + (d2**2)*w2 + (d3**2)*w3)

            # Donor group
            group = eu_mhq_cv.loc[dd <= DIST, 'id']
            stations_reg_group = stations_tab.loc[stations_tab['id'].isin(group)]
            eu_mhq_cv_group = eu_mhq_cv.loc[eu_mhq_cv['id'].isin(group)]

            # Continue adapting the rest of the code here...

    # Set the appropriate names and return the result
    reg_megafloods_list = dict(zip(range(1, 6), reg_meg